In [9]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [10]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.graph_objs as go


from neo4j import GraphDatabase


with driver.session() as session:
   
    result = session.run("MATCH (p:Product)-[:STOCKED_IN]->(c:Category), "
                         "(p)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch) "
                         "RETURN p.name AS product, res.ingredient_name AS ingredientResearch, COLLECT(c.name) AS categories")

    # Convert results to Pandas DataFrame
    data = []
    for record in result:
        data.append({'product': record['product'], 'ingredientResearch': record['ingredientResearch'], 'categories': record['categories']})
    df = pd.DataFrame(data)

    # Solo heat coding of raw material research and product categories
    mlb = MultiLabelBinarizer()
    ingredient_encoded = pd.DataFrame(mlb.fit_transform(df['ingredientResearch'].apply(lambda x: [x])),
                                      columns=mlb.classes_, index=df.index)
    category_encoded = pd.DataFrame(mlb.fit_transform(df['categories']),
                                    columns=mlb.classes_, index=df.index)

    # Splicing of coded raw material studies and product categories
    features = pd.concat([ingredient_encoded, category_encoded], axis=1)

    # Clustering using the K-means algorithm
    kmeans = KMeans(n_clusters=6, random_state=50)
    clusters = kmeans.fit_predict(features)

    # Add the clustering results to the DataFrame
    df['community'] = clusters

    # Create table data
    table_data = []
    for community in sorted(df['community'].unique()):
        products = df[df['community'] == community]['product'].unique().tolist()
        row = [f"<b>community {community}</b>"] + ["<br>".join(products)]
        table_data.append(row)

    
    table = go.Table(
        header=dict(values=["community", "Products"]),
        cells=dict(values=[list(zip(*table_data))[0], list(zip(*table_data))[1]],
                   fill_color=[['#f2f2f2', 'white'] * len(table_data)],
                   align=['center', 'left'],
                   font=dict(size=12),
                   height=30)
    )

    
    layout = go.Layout(title="Product communities Based on Ingredient Research and Category (K-means)",
                       width=1000, height=600)

   
    fig = go.Figure(data=[table], layout=layout)
    fig.show()

c:\Users\apple\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [11]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.graph_objects as go

# ... (data loading and preprocessing code remains the same)

# Clustering using the K-means algorithm
kmeans = KMeans(n_clusters=6, random_state=50)
clusters = kmeans.fit_predict(features)

# Add the clustering results to the DataFrame
df['community'] = clusters

# Filter products in community 3
community_3_df = df[df['community'] == 3]

if len(community_3_df) > 0:  # Check if there are products in community 3
    # One-hot encoding of ingredient research and product categories for community 3
    community_3_ingredient_encoded = pd.DataFrame(mlb.transform(community_3_df['ingredientResearch'].apply(lambda x: [x])), 
                                                  columns=mlb.classes_, index=community_3_df.index)
    community_3_category_encoded = pd.DataFrame(mlb.transform(community_3_df['categories']), 
                                                columns=mlb.classes_, index=community_3_df.index)

    # Concatenate encoded ingredient research and product categories for community 3
    community_3_features = pd.concat([community_3_ingredient_encoded, community_3_category_encoded], axis=1)

    # Clustering using the K-means algorithm for community 3
    sub_kmeans = KMeans(n_clusters=10, random_state=50)  # Adjust the number of clusters as needed
    sub_clusters = sub_kmeans.fit_predict(community_3_features)

    # Update the community labels for the subdivided community 3
    df.loc[df['community'] == 3, 'community'] = sub_clusters + 6  # 6, 7, 8

# Create table data
table_data = []
for community in sorted(df['community'].unique()):
    products = df[df['community'] == community]['product'].unique().tolist()
    row = [f"<b>Community {community}</b>"] + ["<br>".join(products)]
    table_data.append(row)

# Create the table
table = go.Table(
    header=dict(values=["Community", "Products"]),
    cells=dict(values=[list(zip(*table_data))[0], list(zip(*table_data))[1]],
               fill_color=[['#f2f2f2', 'white'] * len(table_data)],
               align=['center', 'left'],
               font=dict(size=12),
               height=30)
)

# Create the layout
layout = go.Layout(title="Product Communities Based on Ingredient Research and Category (K-means)", 
                   width=1000, height=600)

# Create the figure
fig = go.Figure(data=[table], layout=layout)

# Show the figure
fig.show()

c:\Users\apple\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\apple\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning:

unknown class(es) ['', 'Acesulfame Potassium', 'Acidity Regulator 339', 'Anthocyanins', 'Antioxidant 385', 'Apple', 'Ascorbic Acid', 'Aspartame', 'Australian Spring Water', 'Bis-Diglyceryl Polyacyladipate-2', 'Caffeine', 'Calcium Lactate', 'Carbonated Mineral Water', 'Carbonated Water', 'Carboxymethyl Cellulose', 'Carmine', 'Carminic Acid', 'Carotene', 'Carrageenan', 'Citric Acid', 'Clarified Pineapple Juice Concentrate', 'Cochineal', 'Coconut Extract', 'Colouring Agent 150d', 'Cream', 'Diced Peaches', 'Diced Pears', 'Ferric Ferrocyanide', 'Flavour', 'Folate', 'Fruit Peach', 'Full Cream Milk', 'Guar Gum', 'Guava Puree', 'Homogenised Full Cream Milk', 'Iron Oxides', 'Jelly Natu

In [14]:
with driver.session() as session:
    session.run("MATCH (c:Community) DETACH DELETE c")
    session.run("MATCH (p:Product)-[r:BELONGS_TO]->() DELETE r")
    session.run("MATCH (p:Product)-[r:COMMUNITY_TO]->() DELETE r")

    # Run the Cypher query and get the results
    result = session.run("MATCH (p:Product)-[:STOCKED_IN]->(c:Category), "
                         "(p)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(res:IngredientResearch) "
                         "RETURN p.name AS product, res.ingredient_name AS ingredientResearch, COLLECT(c.name) AS categories")

    # Convert results to Pandas DataFrame
    data = []
    for record in result:
        data.append({'product': record['product'], 'ingredientResearch': record['ingredientResearch'], 'categories': record['categories']})
    df = pd.DataFrame(data)

    # Solo heat coding of raw material research and product categories
    mlb = MultiLabelBinarizer()
    ingredient_encoded = pd.DataFrame(mlb.fit_transform(df['ingredientResearch'].apply(lambda x: [x])), columns=mlb.classes_, index=df.index)
    category_encoded = pd.DataFrame(mlb.fit_transform(df['categories']), columns=mlb.classes_, index=df.index)

    # Splicing of coded raw material studies and product categories
    features = pd.concat([ingredient_encoded, category_encoded], axis=1)

    # Use K-means algorithm for clustering, change the number of clusters to 6 and set random_state to 50
    kmeans = KMeans(n_clusters=6, random_state=50)
    clusters = kmeans.fit_predict(features)

    # Add the clustering results to the DataFrame
    df['community'] = clusters

    # Filter products in community 3
    community_3_df = df[df['community'] == 3]
    if len(community_3_df) > 0:
        # One-hot encoding of ingredient research and product categories for community 3
        community_3_ingredient_encoded = pd.DataFrame(mlb.transform(community_3_df['ingredientResearch'].apply(lambda x: [x])), columns=mlb.classes_, index=community_3_df.index)
        community_3_category_encoded = pd.DataFrame(mlb.transform(community_3_df['categories']), columns=mlb.classes_, index=community_3_df.index)

        # Concatenate encoded ingredient research and product categories for community 3
        community_3_features = pd.concat([community_3_ingredient_encoded, community_3_category_encoded], axis=1)

        # Clustering using the K-means algorithm for community 3
        sub_kmeans = KMeans(n_clusters=10, random_state=50)
        sub_clusters = sub_kmeans.fit_predict(community_3_features)

        # Update the community labels for the subdivided community 3
        df.loc[df['community'] == 3, 'community'] = sub_clusters + 6

with driver.session() as session:
    # Update the dictionary of community names and descriptions
    community_info = {
        0: {'name': 'Chips', 'description': 'This community is related to Chips.'},
        1: {'name': 'Noodles', 'description': 'This community is related to Noodles.'},
        2: {'name': 'Cookies', 'description': 'This community is related to Cookies.'},
        4: {'name': 'Bar', 'description': 'This community is related to Bar.'},
        5: {'name': 'Sauce', 'description': 'This community is related to Sauce.'},
        6: {'name': 'Fruit juice drinks', 'description': 'This community is related to Fruit juice drinks.'},
        7: {'name': 'Can and bottle', 'description': 'This community is related to Can and bottle.'},
        8: {'name': 'Fruit cans', 'description': 'This community is related to Fruit cans .'},
        9: {'name': 'Make up', 'description': 'This community is related to make up.'},
        10: {'name': 'Water bottle', 'description': 'This community is related to Water bottle.'},
        11: {'name': 'Fresh Full Cream Milk', 'description': 'This community is related to Fresh Full Cream Milk.'},
        12: {'name': 'Coconut', 'description': 'This community is related to coconut.'},
        13: {'name': 'Oil', 'description': 'This community is related to oil.'},
        14: {'name': 'Vegetable cans', 'description': 'This community is related to vegetable cans.'},
        15: {'name': 'Long life full cream milk', 'description': 'This community is related to long life full cream milk.'}
    }

    # Creating a Community node in Neo4j
    for cluster_id, info in community_info.items():
        query = "MERGE (c:Community {name: $name, description: $description})"
        session.run(query, name=info['name'], description=info['description'])

    # Create a relationship between the Product node and the Community node
    for _, row in df.iterrows():
        product = row['product']
        community_id = row['community']
        
        # Check if the community ID exists in the community_info dictionary
        if community_id in community_info:
            cluster_name = community_info[community_id]['name']
            query = "MATCH (p:Product {name: $product}), (c:Community {name: $cluster_name}) MERGE (p)-[:COMMUNITY_TO]->(c)"
            session.run(query, product=product, cluster_name=cluster_name)
        else:
            print(f"Community ID {community_id} not found for product: {product}")

c:\Users\apple\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\apple\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning:

unknown class(es) ['', 'Acesulfame Potassium', 'Acidity Regulator 339', 'Anthocyanins', 'Antioxidant 385', 'Apple', 'Ascorbic Acid', 'Aspartame', 'Australian Spring Water', 'Bis-Diglyceryl Polyacyladipate-2', 'Caffeine', 'Calcium Lactate', 'Carbonated Mineral Water', 'Carbonated Water', 'Carboxymethyl Cellulose', 'Carmine', 'Carminic Acid', 'Carotene', 'Carrageenan', 'Citric Acid', 'Clarified Pineapple Juice Concentrate', 'Cochineal', 'Coconut Extract', 'Colouring Agent 150d', 'Cream', 'Diced Peaches', 'Diced Pears', 'Ferric Ferrocyanide', 'Flavour', 'Folate', 'Fruit Peach', 'Full Cream Milk', 'Guar Gum', 'Guava Puree', 'Homogenised Full Cream Milk', 'Iron Oxides', 'Jelly Natu

In [ ]:
query = """
CREATE (grains:DashDietCategory {name: "Grains", description: "6–8 servings of grains or grain products"})
CREATE (fruits:DashDietCategory {name: "Fruits", description: "4–5 servings of fruits"})
CREATE (vegetables:DashDietCategory {name: "Vegetables", description: "4–5 servings of vegetables"})
CREATE (lowFatDairy:DashDietCategory {name: "LowFatDairyFoods", description: "2–3 servings of low-fat dairy foods"})
CREATE (fatsAndOils:DashDietCategory {name: "FatsAndOils", description: "2–3 servings of fats and oils"})
CREATE (meatPoultryFish:DashDietCategory {name: "MeatPoultryFish", description: "2 or fewer servings of meat, poultry or fish"})

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:STOCKED_IN]->(:Category)
WHERE toLower(p.name) CONTAINS 'rice' OR any(c IN p.categories WHERE toLower(c) CONTAINS 'rice')
MERGE (grains)-[:Suitable_For]->(p)

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:STOCKED_IN]->(:Category)
WHERE toLower(p.name) CONTAINS 'fruit' OR any(c IN p.categories WHERE toLower(c) CONTAINS 'fruit')
MERGE (fruits)-[:Suitable_For]->(p)

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:STOCKED_IN]->(:Category)
WHERE toLower(p.name) CONTAINS 'vegetable' OR any(c IN p.categories WHERE toLower(c) CONTAINS 'vegetable')
MERGE (vegetables)-[:Suitable_For]->(p)

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:STOCKED_IN]->(:Category)-[:HAS_NUTRITIONAL_INFO]->(ni:NutritionalInformation)
WHERE toLower(p.name) CONTAINS 'dairy' OR any(c IN p.categories WHERE toLower(c) CONTAINS 'dairy')
  AND toFloat(ni.`Fat Total Quantity Per 100g - ValueWord - NIP`) < 3.1
MERGE (lowFatDairy)-[:Suitable_For]->(p)

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:STOCKED_IN]->(:Category)
WHERE toLower(p.name) CONTAINS 'oil' OR any(c IN p.categories WHERE toLower(c) CONTAINS 'oil')
MERGE (fatsAndOils)-[:Suitable_For]->(p)

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:STOCKED_IN]->(:Category)
WHERE toLower(p.name) CONTAINS 'fish' OR toLower(p.name) CONTAINS 'meat' OR toLower(p.name) CONTAINS 'poultry'
  OR any(c IN p.categories WHERE toLower(c) CONTAINS 'fish' OR toLower(c) CONTAINS 'meat' OR toLower(c) CONTAINS 'poultry')
MERGE (meatPoultryFish)-[:Suitable_For]->(p)

WITH grains, fruits, vegetables, lowFatDairy, fatsAndOils, meatPoultryFish

MATCH (p:Product)-[:HAS_RECIPE]->(:Recipe)-[:HAS_NUTRITIONAL_INFO]->(ni:NutritionalInformation)
WITH p, ni, toFloat(ni.Energy_kJ) / 4.184 AS calories
SET p.Calorie = calories

RETURN p.name, p.Calorie

"""

In [ ]:
query = """

// Create the User node and add the description attribute.
CREATE (user1:User {name: 'User 1', totalCalories: 1400, description: 'User 1 would like to shop for 1400 calories total.'})
CREATE (user2:User {name: 'User 2', totalCalories: 2400, description: 'User 2 would like to try a keto diet, total 2400 calories.'})
CREATE (user3:User {name: 'User 3', totalCalories: 2000, description: 'User 3 follows a Gluten-free casein-free diet, totaling 2000 calories per day.'})
WITH user1, user2, user3

// User 1: Random selection of products with up to 1400 total calories
MATCH (p:Product)
WITH p, rand() AS random, user1
ORDER BY random
WITH collect(p) AS products, user1
WITH products, user1, [p IN products WHERE p.Calorie IS NOT NULL] AS filteredProducts
WITH user1, filteredProducts, reduce(totalCalories = 0, p IN filteredProducts | totalCalories + p.Calorie) AS totalCalories
WHERE totalCalories <= user1.totalCalories
MERGE (user1)-[:SELECTED]->(selectedProducts1:SelectedProducts {products: filteredProducts, totalCalories: reduce(totalCalories = 0, p IN filteredProducts | totalCalories + p.Calorie)})
WITH user1, selectedProducts1

// User 2: Randomly selected keto diet products with up to 2400 total calories.
MATCH (user2:User {name: 'User 2'})
MATCH (p:Product)-[:Keto_Diet]->(c:DietaryClustering {name: 'KetoDiet'})
WITH p, rand() AS random, user2
ORDER BY random
WITH collect(p) AS products, user2
WITH products, user2, [p IN products WHERE p.Calorie IS NOT NULL] AS filteredProducts
WITH user2, filteredProducts, reduce(totalCalories = 0, p IN filteredProducts | totalCalories + p.Calorie) AS totalCalories
WHERE totalCalories <= user2.totalCalories
MERGE (user2)-[:SELECTED]->(selectedProducts2:SelectedProducts {products: filteredProducts, totalCalories: reduce(totalCalories = 0, p IN filteredProducts | totalCalories + p.Calorie)})
WITH user2, selectedProducts2

// User 3: Randomly selected Gluten-free casein-free products with no more than 2000 total calories.
MATCH (user3:User {name: 'User 3'})
MATCH (gfcf:GlutenFreeAndCaseinFree {name: 'GlutenFreeAndCaseinFree'})-[:GLUTENFREECASEINFREE_TO]-(p:Product)
WITH p, rand() AS random, user3
ORDER BY random
WITH collect(p) AS products, user3
WITH products, user3, [p IN products WHERE p.Calorie IS NOT NULL] AS filteredProducts
WITH user3, filteredProducts, reduce(totalCalories = 0, p IN filteredProducts | totalCalories + p.Calorie) AS totalCalories
WHERE totalCalories <= user3.totalCalories
MERGE (user3)-[:SELECTED]->(selectedProducts3:SelectedProducts {products: filteredProducts, totalCalories: reduce(totalCalories = 0, p IN filteredProducts | totalCalories + p.Calorie)})
WITH user3, selectedProducts3

// Returns information, selected products and total calories for each user
MATCH (u:User)-[:SELECTED]->(sp:SelectedProducts)
RETURN u.name AS user, u.description AS userDescription, sp.products AS selectedProducts, sp.totalCalories AS totalCalories

"""

In [ ]:
query = """
Final
// Create a User node and add a description attribute.
MERGE (user1:User {name: 'User 1'})
ON CREATE SET user1.totalCalories = 1400, user1.description = 'User 1 would like to shop for 1400 calories total.'
WITH user1

MERGE (user2:User {name: 'User 2'})
ON CREATE SET user2.totalCalories = 2400, user2.description = 'User 2 would like to try a keto diet, total 2400 calories.'
WITH user1, user2

MERGE (user3:User {name: 'User 3'})
ON CREATE SET user3.totalCalories = 2000, user3.description = 'User 3 follows a Gluten-free casein-free diet, totaling 2000 calories per day.'
RETURN user1, user2, user3

// Ensure that the User 1 node already exists 
MATCH (user1:User {name: 'User 1'})

// Delete the SELECTED relationship associated with user1 and the node named SelectedProducts_User1 and all its relationships.
OPTIONAL MATCH (user1)-[r1:SELECTED]->(sp1:SelectedProductsUser1)
DETACH DELETE sp1
DELETE r1

// Pass the user1 variable for subsequent matches
WITH user1

// Delete the SELECTED relationship associated with user1 and the node named SelectedProducts_User1
OPTIONAL MATCH (user1)-[r2:SELECTED]->(sp2:SelectedProduct_User1)
DETACH DELETE sp2
DELETE r2

// Returns the user1 node after the operation has completed
RETURN user1


User2
// 确保 User 2 节点已经存在 
MATCH (user2:User {name: 'User 2'})

// 删除与 user2 相关的 SELECTED 关系以及名为 SelectedProducts_User2 的节点及其所有关系
OPTIONAL MATCH (user2)-[r:SELECTED]->(sp:SelectedProducts_User2)
DETACH DELETE sp
DELETE r

// 传递 user2 变量以用于后续操作
WITH user2

// 随机选择总卡路里不超过 2400 的 Keto Diet 产品并合并到 user2 的选择中
CALL {
    WITH user2
    MATCH (dc:DietaryClustering {name: 'KetoDiet'})-[:Keto_Diet]->(p:Product)
    WHERE p.Calorie IS NOT NULL
    WITH p, rand() AS random
    ORDER BY random
    WITH collect(p) AS products

    CALL {
        WITH products
        WITH 0 AS runningTotal, [] AS selectedProducts, products
        RETURN reduce(accProducts = selectedProducts, prod IN products |
            CASE
                WHEN runningTotal + prod.Calorie <= 2400
                THEN accProducts + prod
                ELSE accProducts  
            END
        ) AS selectedProducts,
        reduce(accTotal = runningTotal, prod IN products |
            CASE 
                WHEN accTotal + prod.Calorie <= 2400
                THEN accTotal + prod.Calorie
                ELSE accTotal
            END
        ) AS runningTotal
    }

    RETURN selectedProducts, runningTotal
}

WITH user2, selectedProducts, runningTotal

// 匹配通用的 SelectedProductUser 节点，并将其名称设置为 SelectedProductsUser2
MERGE (sp:SelectedProductUser {userId: user2.name + '-selected'})
ON CREATE SET sp.name = 'SelectedProductsUser2', sp.productIds = [], sp.totalCalories = runningTotal
ON MATCH SET sp.name = 'SelectedProductsUser2', sp.productIds = [], sp.totalCalories = runningTotal

WITH user2, selectedProducts, sp

// 展开 selectedProducts 列表并逐个将产品ID添加到 sp.productIds 中
UNWIND selectedProducts AS spProduct
SET sp.productIds = sp.productIds + spProduct.productId

// 创建关系链接
MERGE (user2)-[:SELECTED]->(sp)

WITH user2, sp, selectedProducts
UNWIND selectedProducts AS product
MERGE (sp)-[:CONTAINS]->(product)

// 返回结果
RETURN user2, sp.productIds AS selectedProductIds, sp.totalCalories AS totalCalories, size(sp.productIds) AS numberOfProducts


User3
//Ensure that the User 3 node already exists 
MATCH (user3:User {name: 'User 3'})

// Delete the SELECTED relationship associated with user3 and the node named SelectedProducts_User3 and all its relationships.
OPTIONAL MATCH (user3)-[r:SELECTED]->(sp:SelectedProducts)
DETACH DELETE sp
DELETE r

// Pass the user3 variable for subsequent matches
WITH user3

// Randomly select Gluten-free casein-free products with no more than 2,000 total calories and combine them into user3's selection.
CALL {
    WITH user3
    MATCH (gfcf:GlutenFreeAndCaseinFree)-[:GLUTENFREECASEINFREE_TO]-(p:Product)
    WHERE p.Calorie IS NOT NULL
    WITH p, rand() AS random
    ORDER BY random
    WITH collect(p) AS products

    CALL {
        WITH products
        WITH 0 AS runningTotal, [] AS selectedProducts, products
        RETURN reduce(accProducts = selectedProducts, prod IN products |
            CASE
                WHEN runningTotal + prod.Calorie <= 2000
                THEN accProducts + prod
                ELSE accProducts  
            END
        ) AS selectedProducts,
        reduce(accTotal = runningTotal, prod IN products |
            CASE 
                WHEN accTotal + prod.Calorie <= 2000
                THEN accTotal + prod.Calorie
                ELSE accTotal
            END
        ) AS runningTotal
    }

    RETURN selectedProducts, runningTotal
}

WITH user3, selectedProducts, runningTotal

// Creating a new SelectedProducts_User3 node
CREATE (sp:SelectedProducts_User3 {userId: user3.name + '-selected', name: 'SelectedProductsUser3', productIds: [], totalCalories: runningTotal})

WITH user3, selectedProducts, sp

// Expand the selectedProducts list and add the product IDs to sp.productIds one by one.
UNWIND selectedProducts AS spProduct
SET sp.productIds = sp.productIds + spProduct.productId

// Creating Relationship Links
MERGE (user3)-[:SELECTED]->(sp)

WITH user3, sp, selectedProducts
UNWIND selectedProducts AS product
MERGE (sp)-[:CONTAINS]->(product)

// Return results
RETURN user3, sp.productIds AS selectedProductIds, sp.totalCalories AS totalCalories, size(sp.productIds) AS numberOfProducts


"""

In [ ]:

query="""

This is the final one 
// Create a User node and add a description attribute.
MERGE (user1:User {name: 'User 1'})
ON CREATE SET user1.totalCalories = 1400, user1.description = 'User 1 would like to shop for 1400 calories total.'
WITH user1

MERGE (user2:User {name: 'User 2'})
ON CREATE SET user2.totalCalories = 2400, user2.description = 'User 2 would like to try a keto diet, total 2400 calories.'
WITH user1, user2

MERGE (user3:User {name: 'User 3'})
ON CREATE SET user3.totalCalories = 2000, user3.description = 'User 3 follows a Gluten-free casein-free diet, totaling 2000 calories per day.'
RETURN user1, user2, user3

// Ensure that the User 1 node already exists 
MATCH (user1:User {name: 'User 1'})

// Delete the SELECTED relationship associated with user1 and the node named SelectedProducts_User1 and all its relationships.
OPTIONAL MATCH (user1)-[r1:SELECTED]->(sp1:SelectedProductsUser {name: 'SelectedProductsUser1'})
DETACH DELETE sp1
DELETE r1

// Pass the user1 variable for subsequent matches
WITH user1

// Returns the user1 node after the operation has completed
RETURN user1


// Ensure that the User 2 node already exists 
MATCH (user2:User {name: 'User 2'})

// Delete the SELECTED relationship associated with user2 and the node named SelectedProductsUser2 and all its relationships.
OPTIONAL MATCH (user2)-[r2:SELECTED]->(sp2:SelectedProductsUser {name: 'SelectedProductsUser2'})
DETACH DELETE sp2
DELETE r2

// Pass the user2 variable for subsequent matches
WITH user2

// Returns the user2 node after the operation has completed
RETURN user2


// Ensure that the User 3 node already exists 
MATCH (user3:User {name: 'User 3'})

// Delete the SELECTED relationship associated with user3 and the node named SelectedProductsUser3 and all its relationships.
OPTIONAL MATCH (user3)-[r3:SELECTED]->(sp3:SelectedProductsUser {name: 'SelectedProductsUser3'})
DETACH DELETE sp3
DELETE r3

// Pass the user3 variable for subsequent matches
WITH user3

// Returns the user3 node after the operation has completed
RETURN user3


// Ensure that the User 1 node already exists 
MATCH (user1:User {name: 'User 1'})

// Randomly select products with up to 1400 total calories and merge them into user1's selection.
CALL {
    WITH user1
    MATCH (p:Product)
    WHERE p.Calorie IS NOT NULL
    WITH p, rand() AS random
    ORDER BY random
    WITH collect(p) AS products

    CALL {
        WITH products
        WITH 0 AS runningTotal, [] AS selectedProducts, products
        RETURN reduce(accProducts = selectedProducts, prod IN products |
            CASE
                WHEN runningTotal + prod.Calorie <= 1400
                THEN accProducts + prod
                ELSE accProducts  
            END
        ) AS selectedProducts,
        reduce(accTotal = runningTotal, prod IN products |
            CASE 
                WHEN accTotal + prod.Calorie <= 1400
                THEN accTotal + prod.Calorie
                ELSE accTotal
            END
        ) AS runningTotal
    }

    RETURN selectedProducts, runningTotal
}

WITH user1, selectedProducts, runningTotal

// Create or match a generic SelectedProductsUser node and set its name to SelectedProductsUser1
MERGE (sp:SelectedProductsUser {userId: user1.name + '-selected'})
ON CREATE SET sp.productIds = [], sp.totalCalories = runningTotal
ON MATCH SET sp.totalCalories = runningTotal

// set the name of the new selection to SelectedProductsUser1
SET sp.name = 'SelectedProductsUser1'

WITH user1, selectedProducts, sp

// Expand the selectedProducts list and add the product IDs to sp.productIds one by one.
UNWIND selectedProducts AS spProduct
SET sp.productIds = sp.productIds + spProduct.productId

// Create relationship links
MERGE (user1)-[:SELECTED]->(sp)

WITH user1, sp, selectedProducts
UNWIND selectedProducts AS product
MERGE (sp)-[:CONTAINS]->(product)

// Return results
RETURN user1, sp.productIds AS selectedProductIds, sp.totalCalories AS totalCalories, size(sp.productIds) AS numberOfProducts



User2
// Ensure that the User 2 node already exists 
MATCH (user2:User {name: 'User 2'})

// Randomly select Keto Diet products with up to 2400 total calories and merge them into user2's selection.
CALL {
    WITH user2
    MATCH (dc:DietaryClustering {name: 'KetoDiet'})-[:Keto_Diet]->(p:Product)
    WHERE p.Calorie IS NOT NULL
    WITH p, rand() AS random
    ORDER BY random
    WITH collect(p) AS products

    CALL {
        WITH products
        WITH 0 AS runningTotal, [] AS selectedProducts, products
        RETURN reduce(accProducts = selectedProducts, prod IN products |
            CASE
                WHEN runningTotal + prod.Calorie <= 2400
                THEN accProducts + prod
                ELSE accProducts  
            END
        ) AS selectedProducts,
        reduce(accTotal = runningTotal, prod IN products |
            CASE 
                WHEN accTotal + prod.Calorie <= 2400
                THEN accTotal + prod.Calorie
                ELSE accTotal
            END
        ) AS runningTotal
    }

    RETURN selectedProducts, runningTotal
}

WITH user2, selectedProducts, runningTotal

// Match or create generic SelectedProductsUser node and set its name to SelectedProductsUser2
MERGE (sp:SelectedProductsUser {userId: user2.name + '-selected'})
ON CREATE SET sp.productIds = [], sp.totalCalories = runningTotal
ON MATCH SET sp.totalCalories = runningTotal

// set the name of the new selection to SelectedProductsUser2
SET sp.name = 'SelectedProductsUser2'

WITH user2, selectedProducts, sp

// Expand the selectedProducts list and add the product IDs to sp.productIds one by one.
UNWIND selectedProducts AS spProduct
SET sp.productIds = sp.productIds + spProduct.productId

// Create relationship links
MERGE (user2)-[:SELECTED]->(sp)

WITH user2, sp, selectedProducts
UNWIND selectedProducts AS product
MERGE (sp)-[:CONTAINS]->(product)

// Return results
RETURN user2, sp.productIds AS selectedProductIds, sp.totalCalories AS totalCalories, size(sp.productIds) AS numberOfProducts



User3
// Ensure that the User 3 node already exists 
MATCH (user3:User {name: 'User 3'})

// Randomly select Gluten-free casein-free products with no more than 2,000 total calories and merge them into user3's selection.
CALL {
    WITH user3
    MATCH(p:Product)-[:GLUTENFREECASEINFREE_TO]->(gfcf:GlutenFreeAndCaseinFree)
    WHERE p.Calorie IS NOT NULL
    WITH p, rand() AS random
    ORDER BY random
    WITH collect(p) AS products

    CALL {
        WITH products
        WITH 0 AS runningTotal, [] AS selectedProducts, products
        RETURN reduce(accProducts = selectedProducts, prod IN products |
            CASE
                WHEN runningTotal + prod.Calorie <= 2000
                THEN accProducts + prod
                ELSE accProducts  
            END
        ) AS selectedProducts,
        reduce(accTotal = runningTotal, prod IN products |
            CASE 
                WHEN accTotal + prod.Calorie <= 2000
                THEN accTotal + prod.Calorie
                ELSE accTotal
            END
        ) AS runningTotal
    }

    RETURN selectedProducts, runningTotal
}

WITH user3, selectedProducts, runningTotal

// Match or create generic SelectedProductsUser node and set its name to SelectedProductsUser3
MERGE (sp:SelectedProductsUser {userId: user3.name + '-selected'})
ON CREATE SET sp.productIds = [], sp.totalCalories = runningTotal
ON MATCH SET sp.totalCalories = runningTotal

// set the name of the new selection to SelectedProductsUser3
SET sp.name = 'SelectedProductsUser3'

WITH user3, selectedProducts, sp

// Expand the selectedProducts list and add the product IDs to sp.productIds one by one.
UNWIND selectedProducts AS spProduct
SET sp.productIds = sp.productIds + spProduct.productId

// Create relationship links
MERGE (user3)-[:SELECTED]->(sp)

WITH user3, sp, selectedProducts
UNWIND selectedProducts AS product
MERGE (sp)-[:CONTAINS]->(product)

// Return results
RETURN user3, sp.productIds AS selectedProductIds, sp.totalCalories AS totalCalories, size(sp.productIds) AS numberOfProducts


"""